In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import xlsxwriter

def linear_classifier(x_vectors, W) :
	aug_ones = np.ones((x_vectors.shape[0],1),dtype=float)
	#print aug_ones
	#print aug_ones.shape
	#print type(aug_ones)
	xa_vectors = np.append(aug_ones,x_vectors,1)
	#print xa_vectors

	classify_failure = np.dot(xa_vectors, W)
	#classify_failure[2,0] = 0
	#print classify_failure
	results = np.zeros((x_vectors.shape[0],1),dtype=int)
	#print results
	#print classify_failure.shape[1]
	if (classify_failure.shape[1]==1) :
		for i,cl in enumerate(classify_failure) :
			#print i, np.asscalar(cl)
			cl_scal = np.asscalar(cl)
			if cl_scal == 0 :
				print "ERROR: 0 value in output class"
				exit(1)
			results[i,0] = 1 if cl_scal>0 else -1
	else :
		for i,cl in enumerate(classify_failure) :
			max_val = max(cl)
			#print i, cl, max_val
			idxs = np.where(cl == max_val)[0]
			if (len(idxs) == 1) :
				#print idxs
				#print np.asscalar(idxs)
				results[i,0] = np.asscalar(idxs)
			else :
				print "ERROR : several max indexes:",idxs
				exit(1)

	return results


#excel_file = 'short.xlsx'
#excel_file = 'Assignment_2_Data_and_Template.xlsx'
#excel_file = 'short_a4.xlsx'

excel_file = 'Assignment_4_Data_and_Template.xlsx'

print "Reading sheet"
data_frame = pd.read_excel(excel_file,sheetname='Training Data')
print data_frame
df_matrix = data_frame.as_matrix()
print df_matrix

#X = df_matrix[:,:-2].astype(float)
X = df_matrix[:,:-2]
print X
T_type = df_matrix[:,-1:]
T_failure = df_matrix[:,-2:-1]
#T_failure = df_matrix[:,-2:-1].astype(float)

print "Data set"
print X
print "Labels"
print T_failure
print T_type

T_type_kesler = np.zeros((T_type.shape[0],6),dtype=float)

# Converting to Kesler's form
for i,typ in enumerate(T_type):
	T_type_kesler[i,np.asscalar(typ)] = 2.0
T_type_kesler = T_type_kesler-1
print T_type_kesler

# Generating Xa and its pesudo inverse
X0 = np.ones((X.shape[0],1),dtype=float)
print X0
print X0.shape
print type(X0)
Xa = np.append(X0,X,1)
print Xa
Xapinv = np.linalg.pinv(Xa)
print Xapinv

# Calculate the w's
W_failure = np.dot(Xapinv, T_failure)
print "W failure"
print W_failure

W_type = np.dot(Xapinv, T_type_kesler)
'''
W_type = np.array(
[[9,10,11,12],
 [5, 6, 7, 8],
 [1, 2, 3, 4]]
)
'''
print "W type"
print W_type

queries_df = pd.read_excel(excel_file,sheetname='To be classified',skiprows=3)
print queries_df
queries_df_matrix = queries_df.as_matrix()
print queries_df_matrix
X_queries = queries_df_matrix[:,:-2]
print X_queries

#results_failure = linear_classifier(X_queries, W_failure)
#X_queries = X
results_failure = linear_classifier(X_queries, W_failure)
print "Predicted failures"
print results_failure
results_type = linear_classifier(X_queries, W_type)
print "Predicted types"
print results_type

# Test accuracy:
cl_as_failure = linear_classifier(X, W_failure)
print  np.append(T_failure,cl_as_failure,1)

conf_failure = np.zeros((2,2),dtype=int)
for i, true_cl_fail in enumerate(T_failure) :
	tcl = np.asscalar(true_cl_fail)
	cas = np.asscalar(cl_as_failure[i])
	#print i, tcl, cas
	row_idx = (tcl + 1)/2
	col_idx = (cas + 1)/2
	conf_failure[row_idx,col_idx] += 1
print "Confusion failure"
print conf_failure

total_failure = np.sum(conf_failure)
print "total_failure:",total_failure

Accuracy_failure = (conf_failure[0,0]+conf_failure[1,1])/float(total_failure)
print "Accuracy_failure",Accuracy_failure
PPV_failure = conf_failure[1,1]/float(conf_failure[0,1]+conf_failure[1,1])
print "PPV_failure",PPV_failure

cl_as_type = linear_classifier(X, W_type)
print  np.append(T_type,cl_as_type,1)

conf_type = np.zeros((6,6),dtype=int)
for i, true_cl_type in enumerate(T_type) :
	tcl = np.asscalar(true_cl_type)
	cas = np.asscalar(cl_as_type[i])
	#print i, tcl, cas
	conf_type[tcl,cas] += 1
	#row_idx = (tcl + 1)/2
print "Confusion type"
print conf_type

PPV_type = np.zeros(6,dtype=float)
for i in range(len(PPV_type)) :
	#print "--"
	#print conf_type[i,i]
	#print conf_type[:,i]
	p_sum = np.sum(conf_type[:,i])
	#print p_sum
	PPV_type[i] = conf_type[i,i]/float(p_sum)
print "PPV_type",PPV_type
max_ppv_type = max(PPV_type)
print "Max PPV type",max_ppv_type
max_ppv_type_idxs = np.where(PPV_type == max_ppv_type)[0]
if (len(max_ppv_type_idxs) == 1) :
	max_ppv_type_idx = np.asscalar(max_ppv_type_idxs)
else :
	print "ERROR : several max indexes:",max_ppv_type_idxs
	max_ppv_type_idx =max_ppv_type_idxs[0]
	#exit(1)
print "Max PPV type index",max_ppv_type_idx

min_ppv_type = min(PPV_type)
print "Min PPV type",min_ppv_type
min_ppv_type_idxs = np.where(PPV_type == min_ppv_type)[0]
if (len(min_ppv_type_idxs) == 1) :
	min_ppv_type_idx = np.asscalar(min_ppv_type_idxs)
else :
	print "ERROR : several min indexes:",min_ppv_type_idxs
	exit(1)
print "Min PPV type index",min_ppv_type_idx


